In [1]:
!pip install pytest pytest-cov scikit-learn pandas numpy joblib matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.0/244.0 kB 5.6 MB/s eta 0:00:00


In [2]:
from google.colab import files
# téléchargez le fichier depuis mon PC vers Google Colab:
upload = files.upload()


Saving nba_shots_preprocessed_optimise.csv to nba_shots_preprocessed_optimise.csv


# Implémentation du fichier train.py

In [4]:
%%writefile train.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def load_data(file_path):
    """Charge les données prétraitées depuis un fichier CSV"""
    data = pd.read_csv(file_path)
    print(f"Données chargées : {data.shape[0]} lignes, {data.shape[1]} colonnes")
    return data

def split_data(data, test_size=0.2, random_state=42):
    """Divise les données en ensembles d'entraînement et de test"""
    X = data.drop(columns=['Shot Made Flag'])
    y = data['Shot Made Flag']

    X_train, X_test, y_train, y_test = sklearn_train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    print(f"Division des données : {len(X_train)} pour l'entraînement, {len(X_test)} pour le test")
    return X_train, X_test, y_train, y_test

def create_model(n_estimators=100, max_depth=None, random_state=42):
    """Crée un modèle RandomForest pour la classification"""
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state,
        n_jobs=-1  # Utilisation de tous les cœurs disponibles
    )
    print(f"Modèle créé : {type(model).__name__}")
    return model

def train_model(model, X_train, y_train):
    """Entraîne le modèle sur les données d'entraînement"""
    print("Début de l'entraînement du modèle...")
    model.fit(X_train, y_train)
    print("Entraînement terminé")
    return model

def evaluate_model(model, X_test, y_test):
    """Évalue les performances du modèle"""
    y_pred = model.predict(X_test)

    # Calcul des métriques
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, zero_division=0),
        'recall': recall_score(y_test, y_pred, zero_division=0),
        'f1': f1_score(y_test, y_pred, zero_division=0)
    }

    print("Évaluation du modèle :")
    for metric_name, metric_value in metrics.items():
        print(f"{metric_name}: {metric_value:.4f}")

    return metrics

def save_model(model, file_path):
    """Sauvegarde le modèle entraîné"""
    joblib.dump(model, file_path)
    print(f"Modèle sauvegardé dans {file_path}")

def visualize_feature_importance(model, X):
    """Visualise l'importance des caractéristiques"""
    feature_importance = model.feature_importances_
    sorted_idx = np.argsort(feature_importance)

    plt.figure(figsize=(10, 8))
    plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
    plt.yticks(range(len(sorted_idx)), X.columns[sorted_idx])
    plt.title('Importance des caractéristiques')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    print("Graphique d'importance des caractéristiques sauvegardé")

def main():
    """Fonction principale qui exécute le pipeline complet"""
    # Charger les données
    data = load_data('nba_shots_preprocessed_optimise.csv')

    # Diviser les données
    X_train, X_test, y_train, y_test = split_data(data)

    # Créer le modèle
    model = create_model()

    # Entraîner le modèle
    trained_model = train_model(model, X_train, y_train)

    # Évaluer le modèle
    metrics = evaluate_model(trained_model, X_test, y_test)

    # Visualiser l'importance des caractéristiques
    visualize_feature_importance(trained_model, X_train)

    # Sauvegarder le modèle
    save_model(trained_model, 'nba_shot_prediction_model.joblib')

    return trained_model, metrics

if __name__ == "__main__":
    main()


Writing train.py


# Création du fichier de test

In [5]:
%%writefile test_train.py
import pytest
import pandas as pd
import numpy as np
import os
import sys
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ajout du répertoire parent au chemin pour importer train.py
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

# Import du module train
import train

# Chemin vers les données prétraitées
DATA_PATH = 'nba_shots_preprocessed_optimise.csv'
MODEL_PATH = 'test_model.joblib'

@pytest.fixture
def sample_data():
    """Fixture pour charger un échantillon des données pour les tests"""
    # Charger les données
    data = pd.read_csv(DATA_PATH)
    # Prendre un échantillon pour accélérer les tests
    return data.sample(100, random_state=42)

def test_load_data():
    """Test du chargement des données"""
    data = train.load_data(DATA_PATH)

    # Vérification du type de retour
    assert isinstance(data, pd.DataFrame), "Les données chargées ne sont pas un DataFrame"

    # Vérification des colonnes attendues
    expected_columns = [
        'Player ID', 'X Location', 'Y Location', 'Total_Seconds_Remaining',
        'Shot_Type_Encoded', 'Shot_Distance_Meters', 'Shot_Zone_Combined', 'Shot Made Flag'
    ]
    assert all(col in data.columns for col in expected_columns), "Colonnes manquantes dans les données"

    # Vérification du nombre de lignes
    assert len(data) > 0, "Le DataFrame est vide"

def test_split_data(sample_data):
    """Test de la division des données en ensembles d'entraînement et de test"""
    X_train, X_test, y_train, y_test = train.split_data(sample_data)

    # Vérification des types
    assert isinstance(X_train, pd.DataFrame), "X_train n'est pas un DataFrame"
    assert isinstance(X_test, pd.DataFrame), "X_test n'est pas un DataFrame"
    assert isinstance(y_train, pd.Series), "y_train n'est pas une Series"
    assert isinstance(y_test, pd.Series), "y_test n'est pas une Series"

    # Vérification des dimensions
    assert len(X_train) > 0, "X_train est vide"
    assert len(X_test) > 0, "X_test est vide"
    assert len(y_train) > 0, "y_train est vide"
    assert len(y_test) > 0, "y_test est vide"

    # Vérification de la division correcte
    assert len(X_train) + len(X_test) == len(sample_data), "La division n'a pas préservé toutes les données"

    # Vérification de l'absence de la variable cible dans X
    assert 'Shot Made Flag' not in X_train.columns, "X_train contient encore la variable cible"
    assert 'Shot Made Flag' not in X_test.columns, "X_test contient encore la variable cible"

def test_create_model():
    """Test de la création du modèle"""
    model = train.create_model()

    # Vérification que le modèle a les méthodes requises
    assert hasattr(model, 'fit'), "Le modèle n'a pas de méthode fit"
    assert hasattr(model, 'predict'), "Le modèle n'a pas de méthode predict"
    assert hasattr(model, 'predict_proba'), "Le modèle n'a pas de méthode predict_proba"

def test_train_model(sample_data):
    """Test de l'entraînement du modèle"""
    X_train, X_test, y_train, y_test = train.split_data(sample_data)
    model = train.create_model()

    # Entraînement du modèle
    trained_model = train.train_model(model, X_train, y_train)

    # Vérification que le modèle peut faire des prédictions
    predictions = trained_model.predict(X_test)
    assert len(predictions) == len(y_test), "Les prédictions n'ont pas la même longueur que y_test"

    # Vérification des valeurs prédites (doivent être 0 ou 1)
    unique_preds = np.unique(predictions)
    assert set(unique_preds).issubset({0, 1}), f"Valeurs de prédiction inattendues: {unique_preds}"

def test_evaluate_model(sample_data):
    """Test de l'évaluation du modèle"""
    X_train, X_test, y_train, y_test = train.split_data(sample_data)
    model = train.create_model()
    trained_model = train.train_model(model, X_train, y_train)

    # Évaluation du modèle
    metrics = train.evaluate_model(trained_model, X_test, y_test)

    # Vérification du type et du contenu des métriques
    assert isinstance(metrics, dict), "Les métriques ne sont pas retournées sous forme de dictionnaire"

    # Vérification de la présence des métriques clés
    expected_metrics = ['accuracy', 'precision', 'recall', 'f1']
    for metric in expected_metrics:
        assert metric in metrics, f"La métrique '{metric}' est absente"
        assert 0 <= metrics[metric] <= 1, f"La métrique '{metric}' n'est pas entre 0 et 1"

def test_save_model(sample_data):
    """Test de la sauvegarde et du chargement du modèle"""
    X_train, X_test, y_train, y_test = train.split_data(sample_data)
    model = train.create_model()
    trained_model = train.train_model(model, X_train, y_train)

    try:
        # Sauvegarde du modèle
        train.save_model(trained_model, MODEL_PATH)

        # Vérification que le fichier existe
        assert os.path.exists(MODEL_PATH), "Le fichier du modèle n'a pas été créé"

        # Chargement du modèle et vérification qu'il fonctionne
        loaded_model = joblib.load(MODEL_PATH)
        assert hasattr(loaded_model, 'predict'), "Le modèle chargé n'a pas de méthode predict"

        # Vérification que le modèle chargé fait les mêmes prédictions
        original_preds = trained_model.predict(X_test)
        loaded_preds = loaded_model.predict(X_test)
        assert np.array_equal(original_preds, loaded_preds), "Les prédictions sont différentes après chargement"

    finally:
        # Nettoyage
        if os.path.exists(MODEL_PATH):
            os.remove(MODEL_PATH)

def test_end_to_end():
    """Test de bout en bout du pipeline d'entraînement"""
    # Chargement et préparation des données
    data = train.load_data(DATA_PATH)
    X_train, X_test, y_train, y_test = train.split_data(data)

    # Création et entraînement du modèle
    model = train.create_model()
    trained_model = train.train_model(model, X_train, y_train)

    # Évaluation du modèle
    metrics = train.evaluate_model(trained_model, X_test, y_test)

    # Vérification de la performance du modèle
    min_accuracy = 0.5  # Exemple de seuil minimal
    assert metrics['accuracy'] > min_accuracy, f"Accuracy trop basse: {metrics['accuracy']}"

    # Sauvegarde et vérification du modèle
    try:
        train.save_model(trained_model, MODEL_PATH)
        assert os.path.exists(MODEL_PATH), "Le modèle n'a pas été sauvegardé"
    finally:
        if os.path.exists(MODEL_PATH):
            os.remove(MODEL_PATH)


Writing test_train.py


In [6]:

# Exécuter les tests
!python -m pytest train.py -v
!python -m pytest test_train.py -v

# Pour obtenir un rapport de couverture de code
!python -m pytest test_train.py --cov=train -v


============================= test session starts ==============================
platform linux -- Python 3.11.12, pytest-8.3.5, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: cov-6.1.1, anyio-4.9.0, langsmith-0.3.34, typeguard-4.4.2
collected 0 items                                                              

============================ no tests ran in 2.36s =============================
============================= test session starts ==============================
platform linux -- Python 3.11.12, pytest-8.3.5, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: cov-6.1.1, anyio-4.9.0, langsmith-0.3.34, typeguard-4.4.2
collected 7 items                                                              

test_train.py::test_load_data PASSED                                     [ 14%]
test_train.py::test_split_data PASSED                                    [ 28%]
test_train.py::test_create_model PASSED                

# Points clés des tests
Les tests unitaires couvrent tous les aspects importants du processus d'entraînement :

Chargement des données : Vérifie que les données sont correctement chargées et ont la structure attendue.

Division des données : Vérifie que les données sont correctement divisées en ensembles d'entraînement et de test.

Création du modèle : Vérifie que le modèle créé a les méthodes nécessaires (fit, predict, predict_proba).

Entraînement du modèle : Vérifie que le modèle peut être entraîné et faire des prédictions.

Évaluation du modèle : Vérifie que les métriques de performance sont correctement calculées.

Sauvegarde et chargement du modèle : Vérifie que le modèle peut être sauvegardé et rechargé avec les mêmes performances.

Test de bout en bout : Vérifie que le pipeline complet fonctionne correctement, de la préparation des données à l'évaluation du modèle.

Ces tests garantissent la robustesse et la fiabilité de notre module d'entraînement dans le cadre du projet MLOps de prédiction des tirs réussis en NBA.